In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import sqlite3

# Establishing the database connection
conn = sqlite3.connect('pv_meteo_data.db')

# Queries for each building
queries = {
    "Dpt_1_2_PV": "SELECT datetime, Prod_kW, GHI, BNI, DHI, DBT, RH, Ws10, Wd10, Patmo, Rainfall FROM Dpt_1_2_PV_Meteo",
    "ENERPOS_PV": "SELECT datetime, Prod_kW, GHI, BNI, DHI, DBT, RH, Ws10, Wd10, Patmo, Rainfall FROM ENERPOS_PV_Meteo",
    "ESIROI_PV": "SELECT datetime, Prod_kW, GHI, BNI, DHI, DBT, RH, Ws10, Wd10, Patmo, Rainfall FROM ESIROI_PV_Meteo"
}

# Function to create sequences for LSTM
def create_sequences(data, past_steps=144, future_steps=25920):  # Adjusted future_steps for 6 months
    X, y = [], []
    for i in range(past_steps, len(data) - future_steps):
        X.append(data[i - past_steps:i])
        y.append(data[i:i + future_steps, 0])
    return np.array(X), np.array(y)

# DataFrame to hold all predictions
all_predictions = pd.DataFrame()

# Loop through each building to train a separate model
for building_name, query in queries.items():
    print(f"\nTraining model for {building_name}...")

    # Fetch data
    df = pd.read_sql_query(query, conn, parse_dates=['datetime'])
    
    # Data Preprocessing
    scaler = RobustScaler()
    df.iloc[:, 1:] = scaler.fit_transform(df.iloc[:, 1:])  # Scale all except datetime
    
    # Feature Engineering
    df['hour'] = df['datetime'].dt.hour
    df['day_of_week'] = df['datetime'].dt.dayofweek
    df['month'] = df['datetime'].dt.month
    df['day_of_year'] = df['datetime'].dt.dayofyear

    # Creating sequences
    features = df.drop(columns=['datetime']).values
    future_steps = 25920  # Define future_steps for 6 months forecast
    X, y = create_sequences(features, past_steps=144, future_steps=future_steps)

    # Split data into training and test sets
    train_size = int(len(X) * 0.7)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # LSTM Model
    model = Sequential([
        LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
        Dropout(0.5),
        BatchNormalization(),
        LSTM(64, return_sequences=False),
        Dropout(0.5),
        Dense(future_steps)
    ])

    # Compile the model
    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10)

    # Model Training
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr])

    # Model Evaluation
    y_pred = model.predict(X_test).flatten()
    mse = mean_squared_error(y_test.flatten(), y_pred)
    mae = mean_absolute_error(y_test.flatten(), y_pred)
    r2 = r2_score(y_test.flatten(), y_pred)
    print(f'MSE: {mse:.3f}, MAE: {mae:.3f}, R-squared: {r2:.3f}')
    
    # Generate datetime for test data
    test_dates = pd.date_range(start=df['datetime'].iloc[train_size + 144], periods=len(y_test.flatten()), freq='10T')

    # Create DataFrame to hold prediction results with timestamps
    results_df = pd.DataFrame({'Datetime': test_dates, 'Actual': y_test.flatten(), 'Predicted': y_pred, 'Building': building_name})
    
    # Append to all_predictions DataFrame
    all_predictions = pd.concat([all_predictions, results_df])

    # Save the results to CSV
    results_df.to_csv(f'{building_name}_predictions.csv', index=False)

    # Plotting actual vs predicted for each building
    plt.figure(figsize=(12, 6))
    plt.plot(test_dates, y_test.flatten(), label='Actual Production', color='blue')
    plt.plot(test_dates, y_pred, label='Predicted Production', color='red', linestyle='--')
    plt.title(f'Predicted vs Actual PV Production for {building_name}')
    plt.xlabel('Date')
    plt.ylabel('PV Production (kW)')
    plt.legend()
    plt.grid(True)
    plt.show()

# Close the database connection
conn.close()
